# Fermionic Dimer

In [ ]:
include("../src/KadanoffBaym.jl")
using OrdinaryDiffEq # for ODEProblem and solve
using RecursiveArrayTools # For ArrayPartition
using PyPlot
using PyCall
qt = pyimport("qutip")
np = pyimport("numpy")

PyPlot.matplotlib.rc("text", usetex=true)
PyPlot.matplotlib.rc("font", family="serif", size=16)

## Model

### Non-Hermitian Hamiltonian:

$$
    \hat{H} = \left(\varepsilon_1 -i\gamma/2\right) c^\phantom{\dagger}_1 c^{\dagger}_1 + \varepsilon_2 c^{\dagger}_2 c^\phantom{\dagger}_2 + J \left(c^{\dagger}_1 c^\phantom{\dagger}_2 + c^{\dagger}_2 c^\phantom{\dagger}_1\right)
$$

### Master equation:

$$
    \partial_{t} \hat{\rho}=-i\left[\hat{H} \hat{\rho}-\hat{\rho} \hat{H}^{
    \dagger}\right]+\gamma c^{\dagger}_1 \hat{\rho} c^\phantom{\dagger}_1
$$

### Equations of motion

#### Vertical Time.

\begin{align}\begin{split}
     ...
\end{split}\end{align}

#### Horizontal Time.

\begin{align}\begin{split}
    ...
\end{split}\end{align}

#### Equal-Time.

\begin{align}\begin{split}
    ...
\end{split}\end{align}

## Solving

In [ ]:
# time parameters
T = 1.0
dt = 2^-8 # this algorithm requires a fixed time
n = Int(1/dt) 

# Non-Hermitian Hamiltonian and jump operator
ε₁ = 0.0
ε₂ = 0.0
J = 2*pi*1.0
γ = 10.0
h = ComplexF64[ε₁ - 0.5im * γ J; J ε₂];

In [ ]:
stop = Int(n/2) + 1 

times = range(0, length=n + 1, stop=T) |> collect;

# defining the tau times
times_tau = 2 .* vcat([-times[stop - (k - 1)] for k in 1:stop - 1], times[1:stop]);

## QuTiP benchmark

### Jordan-Wigner transformation

$$
\begin{align*}
    c^\phantom{\dagger}_1 &\to \sigma^-_1,\\
    c^{\dagger}_1 &\to \sigma^+_1.\\
\end{align*}
$$

$$
\begin{align*}
    c^\phantom{\dagger}_2 &\to (-\sigma^z_1)\otimes\sigma^-_2,\\
    c^{\dagger}_2 &\to (-\sigma^z_1)\otimes\sigma^+_2.\\
\end{align*}
$$

In [ ]:
# initial state
psi0_list = [qt.basis(2, 1), qt.basis(2, 0)]
psi0 = qt.tensor(psi0_list)

# define annihilation operators
c_1_list = [qt.destroy(2), qt.qeye(2)]
c_1  = qt.tensor(c_1_list)    
c_2_list = [qt.destroy(2) * qt.destroy(2).dag() - qt.destroy(2).dag() * qt.destroy(2), qt.destroy(2)]
# c_2_list = [qt.qeye(2), qt.destroy(2)]
c_2  = qt.tensor(c_2_list)    

# Hamiltonian
H  = J * c_1.dag() * c_2
H += H.dag();
H += ε₁ * c_1.dag() * c_1 + ε₂ * c_2.dag() * c_2; # do not give qutip a non-Hermitian Hamiltonian

observables = [c_1.dag()*c_1, c_2.dag()*c_2];

In [ ]:
j_ops = [sqrt(γ) * c_1.dag()] # qutip only needs the jump operator

# quickly solve once for observables
me = qt.mesolve(H, psi0, times, j_ops, observables)

# solve for the time-dependent density matrix
t_sols = qt.mesolve(H, psi0, times, j_ops); # t_sols.states returns density matrices

#### Two times

Calculate $\langle c^\dagger_1(t') c^\phantom{\dagger}_1(t)\rangle$ and $\langle c^\phantom{\dagger}_1(t')c^\dagger_1(t) \rangle$, where $t' = t + \tau$.

In [ ]:
tau_t_sols = Dict()
for k in 1:length(t_sols.states)
    tau_t_sols[k] = qt.mesolve(H, c_1 * t_sols.states[k], times, j_ops).states
end

c_1_dag_c_1 = zeros(ComplexF64, length(t_sols.states), length(t_sols.states))
for k in 1:length(t_sols.states)
    for l in 1:length(t_sols.states)
        c_1_dag_c_1[k, l] = (c_1.dag() * tau_t_sols[k][l]).tr()    
    end
end

tau_t_sols = Dict()
for k in 1:length(t_sols.states)
    tau_t_sols[k] = qt.mesolve(H, c_1.dag() * t_sols.states[k], times, j_ops).states
end

c_1_c_1_dag = zeros(ComplexF64, length(t_sols.states), length(t_sols.states))
for k in 1:length(t_sols.states)
    for l in 1:length(t_sols.states)
        c_1_c_1_dag[k, l] = (c_1 * tau_t_sols[k][l]).tr()    
    end
end

In [ ]:
tau_t_sols = Dict()
for k in 1:length(t_sols.states)
    tau_t_sols[k] = qt.mesolve(H, c_2 * t_sols.states[k], times, j_ops).states
end

c_2_dag_c_2 = zeros(ComplexF64, length(t_sols.states), length(t_sols.states))
for k in 1:length(t_sols.states)
    for l in 1:length(t_sols.states)
        c_2_dag_c_2[k, l] = (c_2.dag() * tau_t_sols[k][l]).tr()    
    end
end

tau_t_sols = Dict()
for k in 1:length(t_sols.states)
    tau_t_sols[k] = qt.mesolve(H, c_2.dag() * t_sols.states[k], times, j_ops).states
end

c_2_c_2_dag = zeros(ComplexF64, length(t_sols.states), length(t_sols.states))
for k in 1:length(t_sols.states)
    for l in 1:length(t_sols.states)
        c_2_c_2_dag[k, l] = (c_2 * tau_t_sols[k][l]).tr()    
    end
end

In [ ]:
# reshape the above array to fit into our two-time "matrix" structure 
# see the plot below for illustration
unskewed_c_1_dag_c_1 = zeros(ComplexF64, length(t_sols.states), 2*length(t_sols.states) - 1)
for (k, x) in enumerate([c_1_dag_c_1[k, :] for k in 1:length(t_sols.states)])
    for (l, y) in enumerate(x)
        ind = k + l - 1 # verify the -1 relative to the original python code
        unskewed_c_1_dag_c_1[k, ind] = y  
    end
end

unskewed_c_1_c_1_dag = zeros(ComplexF64, length(t_sols.states), 2*length(t_sols.states) - 1)
for (k, x) in enumerate([c_1_c_1_dag[k, :] for k in 1:length(t_sols.states)])
    for (l, y) in enumerate(x)
        ind = k + l - 1 # verify the -1 relative to the original python code
        unskewed_c_1_c_1_dag[k, ind] = y  
    end
end

In [ ]:
# reshape the above array to fit into our two-time "matrix" structure 
# see the plot below for illustration
unskewed_c_2_dag_c_2 = zeros(ComplexF64, length(t_sols.states), 2*length(t_sols.states) - 1)
for (k, x) in enumerate([c_2_dag_c_2[k, :] for k in 1:length(t_sols.states)])
    for (l, y) in enumerate(x)
        ind = k + l - 1 # verify the -1 relative to the original python code
        unskewed_c_2_dag_c_2[k, ind] = y  
    end
end

unskewed_c_2_c_2_dag = zeros(ComplexF64, length(t_sols.states), 2*length(t_sols.states) - 1)
for (k, x) in enumerate([c_2_c_2_dag[k, :] for k in 1:length(t_sols.states)])
    for (l, y) in enumerate(x)
        ind = k + l - 1 # verify the -1 relative to the original python code
        unskewed_c_2_c_2_dag[k, ind] = y  
    end
end

In [ ]:
figure(figsize=(6, 2))
subplot(121)
imshow(real(c_1_dag_c_1), cmap="plasma")

subplot(122)
imshow(real(unskewed_c_1_dag_c_1), cmap="plasma")

tight_layout()

## Plotting

In [ ]:
c_1_dag_c_1_tau = [unskewed_c_1_dag_c_1[stop - (k - 1), stop + (k - 1)] for k in 1:stop]
c_1_dag_c_1_tau_reversed = [c_1_dag_c_1_tau[stop - (k - 1)] for k in 1:stop]
c_1_dag_c_1_tau_full = vcat(conj(c_1_dag_c_1_tau_reversed[1:end-1]), c_1_dag_c_1_tau);

c_1_c_1_dag_tau = [unskewed_c_1_c_1_dag[stop - (k - 1), stop + (k - 1)] for k in 1:stop]
c_1_c_1_dag_tau_reversed = [c_1_c_1_dag_tau[stop - (k - 1)] for k in 1:stop]
c_1_c_1_dag_tau_full = vcat(conj(c_1_c_1_dag_tau_reversed[1:end-1]), c_1_c_1_dag_tau);

In [ ]:
c_2_dag_c_2_tau = [unskewed_c_2_dag_c_2[stop - (k - 1), stop + (k - 1)] for k in 1:stop]
c_2_dag_c_2_tau_reversed = [c_2_dag_c_2_tau[stop - (k - 1)] for k in 1:stop]
c_2_dag_c_2_tau_full = vcat(conj(c_2_dag_c_2_tau_reversed[1:end-1]), c_2_dag_c_2_tau);

c_2_c_2_dag_tau = [unskewed_c_2_c_2_dag[stop - (k - 1), stop + (k - 1)] for k in 1:stop]
c_2_c_2_dag_tau_reversed = [c_2_c_2_dag_tau[stop - (k - 1)] for k in 1:stop]
c_2_c_2_dag_tau_full = vcat(conj(c_2_c_2_dag_tau_reversed[1:end-1]), c_2_c_2_dag_tau);

In [ ]:
figure(figsize=(12, 3))
subplot(121)
plot(times, me.expect[1], c="k", ls="--")
plot(times, me.expect[2], c="k", ls="--")
plot(times, [real(unskewed_c_1_dag_c_1[k, k]) for k in 1:n+1], c="r", ls=":", lw=3.0, alpha=0.5)
xlim(0, T)
ylim(0, 1)
xlabel("\$t\$")

subplot(122)
plot(times_tau, c_1_c_1_dag_tau_full - c_1_dag_c_1_tau_full, c="k", ls="--")
plot(times_tau, c_2_c_2_dag_tau_full - c_2_dag_c_2_tau_full, c="k", ls="--")
xlim(-T, T)
xlabel("\$\\tau\$")

tight_layout()
# savefig("test.pdf")